In [ ]:
#| default_exp pheno

In [ ]:
#| export
import jax
import jax.numpy as jnp
from chewc.structs import Population, Trait, _flatten_gather_chr_locus
from typing import Tuple

# compute_dosage remains the same...
@jax.jit
def compute_dosage(population: Population) -> jnp.ndarray:
    """Computes allele dosage. Shape: (n_ind, n_chr, n_loci)"""
    return jnp.sum(population.geno, axis=2)




@jax.jit
def calculate_phenotypes(
    key: jax.Array,
    population: Population,
    trait: Trait,
    heritability: jnp.ndarray,  # Narrow-sense h^2, shape: (n_traits,)
) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """
    Return (phenotypes, true_breeding_values).
    Calculates phenotypes by modeling G = A + D.
    """
    # --- 1. Get QTL dosages ---
    dosage = jnp.sum(population.geno, axis=2, dtype=jnp.int32)
    qtl_dosage = _flatten_gather_chr_locus(
        dosage, trait.qtl_chromosome, trait.qtl_position
    ).astype(jnp.float32)

    # --- 2. Calculate Genetic Components (A and D) ---
    # True Breeding Value (A) is the purely additive component
    true_breeding_value = qtl_dosage @ trait.qtl_effects

    # Dominance Deviation Value (D)
    dominance_dosage = (qtl_dosage == 1).astype(jnp.float32)
    dominance_value = dominance_dosage @ trait.qtl_dominance_effects

    # Total Genotypic Value (G = intercept + A + D)
    genotypic_value = trait.intercept + true_breeding_value + dominance_value

    # --- 3. Calculate Variance Components to derive Environmental Variance (VE) ---
    add_var = jnp.var(true_breeding_value, axis=0)  # VA
    dom_var = jnp.var(dominance_value, axis=0)      # VD

    # Assuming narrow-sense heritability h^2 = VA / VP = VA / (VA + VD + VE)
    # We can derive VE = (VA / h^2) - VA - VD
    h2 = jnp.clip(heritability.astype(jnp.float32), 1e-8, 1.0 - 1e-8)
    env_var = jnp.maximum(0.0, (add_var / h2) - add_var - dom_var)

    # --- 4. Generate Final Phenotypes ---
    noise = jax.random.normal(key, genotypic_value.shape)
    phenotypes = genotypic_value + noise * jnp.sqrt(env_var)

    # Return phenotypes and the true breeding value (for selection and tracking gain)
    return phenotypes, true_breeding_value


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()